In [ ]:
import os
import sys
import pandas as pd

current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)
    
from src import feature_extractor as fe

# --- 1. 파일 경로 설정 및 유효성 검사 ---
# feature_extractor.py의 __main__ 블록에서 사용된 경로를 재사용합니다.
# 이 파일들이 실제로 존재해야 코드가 정상적으로 실행됩니다.
# (스크립트 실행 전, 'data/raw/1120 sensor_data/' 경로에 파일들을 준비해야 합니다.)

BASE_DIR = 'data/raw/1120_S1_yellow'

sensor_files = {
    'accel_gyro': os.path.join(BASE_DIR, '1120_accel_gyro_fluctuating_yellow.csv'),
    'pressure': os.path.join(BASE_DIR, '1120_pressure_fluctuating_yellow.csv'),
}

valid_files = {k: v for k, v in sensor_files.items() if os.path.exists(v)}

if not valid_files:
    print("❌ 경고: 필요한 원본 센서 파일이 발견되지 않았습니다.")
    print(f"다음 경로에 파일이 있는지 확인해주세요: {BASE_DIR}")
    print("더미 파일을 생성하여 테스트를 진행하거나, 실제 파일을 준비해야 합니다.")
    # 실제 파일이 없는 경우, 이 시점에서 스크립트를 종료할 수 있습니다.
    # raise FileNotFoundError("원본 센서 파일이 부족하여 특징 추출을 진행할 수 없습니다.")


# --- 2. 특징 추출 설정 ---
# feature_extractor.py에 정의된 상수를 사용합니다.
WINDOW_SIZE = fe.WINDOW_SIZE      # 10.0 초
WINDOW_OVERLAP = fe.WINDOW_OVERLAP # 5.0 초
RESAMPLE_RATE = '78.125ms' # feature_extractor.py의 __main__에서 사용된 값

# --- 3. 특징 추출 함수 호출 ---
print("=" * 50)
print(f"📊 특징 추출 시작")
print(f"  윈도우 크기: {WINDOW_SIZE}s, 오버랩: {WINDOW_OVERLAP}s")
print(f"  동기화 주기: {RESAMPLE_RATE}")
print("=" * 50)

if valid_files:
    features_df = fe.process_multi_sensor_files(
        valid_files,
        resample_rate=RESAMPLE_RATE,
        window_size=WINDOW_SIZE,
        window_overlap=WINDOW_OVERLAP
    )

    # --- 4. 결과 확인 및 저장 ---
    if not features_df.empty:
        output_dir = fe.OUTPUT_DIR
        output_filename = "1120_features_S1_yellow.csv"
        output_path = os.path.join(output_dir, output_filename)
        
        # 출력 디렉토리가 없으면 생성 (fe.py에서 이미 하지만, 안전을 위해)
        os.makedirs(output_dir, exist_ok=True)
        
        features_df.to_csv(output_path, index=False)
        
        print("\n" + "=" * 50)
        print("✅ 특징 추출 완료")
        print(f"  저장 경로: {output_path}")
        print(f"  결과 Shape: {features_df.shape}")
        print(f"  추출된 특징 개수: {features_df.shape[1] - 3}")
        print("=" * 50)
        
        # 추출된 특징의 일부 확인
        print("\n📋 추출된 특징 데이터프레임 (상위 5개 행):")
        display(features_df.head())
        
        # 컬럼 목록 확인 (선택 사항)
        print("\n⚙️  전체 특징 컬럼 목록:")
        print(features_df.columns.tolist())
        
    else:
        print("\n❌ 특징 추출 결과가 비어 있습니다. 원인: 데이터 부족 또는 처리 오류.")
else:
    print("\n⚠️ 특징 추출을 건너뜁니다. 유효한 입력 파일이 없습니다.")